In [ ]:
!pip install -U spacy -q

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp = spacy.blank("en")
db = DocBin()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/MTP after 15/Annotations/Annotations of 59 train datas.json')
TRAIN_DATA = json.load(f)

In [ ]:
TRAIN_DATA

{'classes': ['COURT OF JUDGEMENT',
  'PARTIES',
  'DATE OF JUDGEMENT',
  'AUTHOR',
  'JUDGE'],
 'annotations': [['MS. Konkan Railway Corporation\r\nvs WS. Rani Construction Pvt: Ltd on 30 January; 2002\r\nSupreme Court of India\r\nM/S. Konkan Railway Corporation\r\nvs M/S. Rani Construction Pvt: Ltd on 30 January, 2002\r\nAuthor: B I.\r\nBench: Cji, Syed Shah Quadri; U. C. Banerjee, SN. Variava, Shivaraj V. Patil\r\nCASE NO _\r\nAppeal (civil) 5880-5889\r\nof\r\n1997\r\nPETITIONER:\r\nM/S\r\nKONKAN RAILWAY CORPORATION LTD .\r\n& ANR _\r\nVs\r\nRESPONDENT\r\nM/S.\r\nRANI CONSTRUCTION PVT. LTD _\r\nDATE OF JUDGMENT:\r\n30/01/2002\r\nBENCH:\r\nCJI,\r\nSYED SHAH MOHAMMED QUADRI _\r\nU,\r\nC. BANERJEE, S.N\r\nVARIAVA & SHIVARAJ V_\r\nPATIL\r\nJUDGMENT =\r\nWITHC.A. Nos. 713-714/1999, C.A No. 715/1999, C.A No. 716/1999, C.A. Nos 2037-2040/1999,\r\nC.A No. 2041/1999,C.A Nos. 2042-2044/1999, C.A No. 4311/1999, C.A No. 4312/1999, CA No.\r\n4324/1999, CA. No. 4356/1999, CA. No. 7304/1999 and CA 

Train test split

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(TRAIN_DATA, test_size=0.2)

KeyError: ignored

In [ ]:
len(train)

47

In [ ]:
for text, annot in tqdm(train):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
           print("Skipping entity")
        else:
           ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

  0%|          | 0/2 [00:00<?, ?it/s]


ValueError: ignored

In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2022-08-23 14:27:33.477689: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2022-08-23 14:27:43.216171: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-08-23 14:27:43,959] [INFO] Set up nlp object from config
[2022-08-23 14:27:43,972] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-08-23 14:27:43,977] [INFO] Created vocabulary
[2022-08-23 14:27:43,980] [INFO] Finished initializing nlp object
[2022-08-23 14:27:45,149] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    197.27    0.00    0.00    0.00    0.00
  4     200       3132

In [ ]:
nlp_ner = spacy.load("/content/model-last")

In [ ]:
nlp_ner

In [ ]:
train

[['\r\n\r\n\r\nMS Mahanagar Telephone Nigam\r\nvs MS H.S Construction Co on 9 October, 2019\r\nDelhi District Court\r\nM/S Mahanagar Telephone Nigam\r\nvs M/S H.S Construction Co on 9 October, 2019\r\nIn THE COURT OF SH HARGURVARINDER SINGH JAGGI _\r\nADDL _\r\nDISTRICT JUDGE\r\n02 _\r\nSOUTH WEST DISTRICT ,\r\nDWARKA COURTS , DELHI\r\nOMP   (COMM)\r\nNo\r\n52/2019\r\nCNR No. DLSWO10025202017\r\nIN THE MATTER OF\r\n1\r\nM/s Mahanagar Telephone Nigam Limited\r\nThe General Manager (Rohini)\r\nMTNL >_\r\nSanchar Parisar, Sec.III, Rohini\r\nNew Delhi -110085\r\nPetitioner\r\nV_\r\n1\r\nM/s H.S.Construction Co\r\nA-1/B-6 , Local Shopping Centre\r\nJanakpuri,\r\nNew Delhi\r\n110058\r\nRespondent\r\nDate f filing of petition:\r\n14.03.2017\r\nDate of judgment reserved:\r\n28 . 08.2019\r\nDate of pronouncement of judgment:\r\n09.10.2019\r\nJUDGMENT\r\n1.A\r\npetition under Section 34 of the Arbitration & Conciliation Act; 1996 (hereinafter "the Act") has\r\nbeen preferred by the petitioner, P

In [ ]:
import pickle

In [ ]:
with open('model1_pickle', 'wb') as f:
  pickle.dump(nlp_ner, f)

In [ ]:
with open('model_pickle', 'rb') as f:
  mp = pickle.load(f)

In [ ]:
with open("/content/T76.txt") as f1:
  x = f1.read()

In [ ]:
doc1 = nlp_ner(x)

In [ ]:
spacy.displacy.render(doc1, style="ent", jupyter=True)